---
## Visualization 1: Market Cap Vs. Stock Prices Over Time By Company and Trading Volume

In [12]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Global color palette for all visualizations
CUSTOM_COLORS = [
    '#d73027',  # red
    '#f46d43',  # coral
    '#fdae61',  # orange-tan
    '#74add1',  # muted blue
    '#4575b4',  # medium blue
    '#313695'   # deep navy
]

# Use this palette for all discrete colors
px.defaults.color_discrete_sequence = CUSTOM_COLORS

# Use the same colors as a continuous scale 
px.defaults.color_continuous_scale = [
    (0.0, '#d73027'),   
    (0.2, '#f46d43'),  
    (0.4, '#fdae61'),   
    (0.6, '#74add1'),  
    (0.8, '#2c7bb6'),
    (1.0, '#313695')    
]

# Load the data
highs = pd.read_csv('Highs.csv')
lows = pd.read_csv('Lows.csv')
macro = pd.read_csv('merged_data(FRED)_new.csv')
volumes = pd.read_csv('monthly_average_volumes.csv')
market_caps = pd.read_csv('monthly_market_caps.csv')

In [13]:
# Convert dates to datetime
highs['Date'] = pd.to_datetime(highs['Date'])
lows['Date'] = pd.to_datetime(lows['Date'])
volumes['Date'] = pd.to_datetime(volumes['Date'])
market_caps['Date'] = pd.to_datetime(market_caps['Date'])

# Get company columns (all columns except Date)
company_cols = [col for col in highs.columns if col != 'Date']

# Calculate mid prices
mid_prices = highs.copy()
for col in company_cols:
    mid_prices[col] = (highs[col] + lows[col]) / 2


In [14]:
# Transform data to long format for visualization
# Mid prices
mid_prices_long = mid_prices.melt(
    id_vars=['Date'],
    var_name='Company',
    value_name='Mid_Price'
)

# Volumes
volumes_long = volumes.melt(
    id_vars=['Date'],
    var_name='Company',
    value_name='Volume'
)

# Market caps
market_caps_long = market_caps.melt(
    id_vars=['Date'],
    var_name='Company',
    value_name='Market_Cap'
)

In [15]:
# Merge all data together
viz_data = mid_prices_long.merge(volumes_long, on=['Date', 'Company'], how='inner')
viz_data = viz_data.merge(market_caps_long, on=['Date', 'Company'], how='inner')

# Remove rows with missing values
viz_data = viz_data.dropna(subset=['Mid_Price', 'Market_Cap', 'Volume'])

# Create year-month column for animation
viz_data['Year_Month'] = viz_data['Date'].dt.strftime('%Y-%m')
viz_data['Year'] = viz_data['Date'].dt.year


In [16]:
# Create the animated bubble chart
# X-axis: Market Cap
# Y-axis: Stock Price (Mid-Price)
# Bubble Size: Volume
# Color: Industry/Subsector
# Aggregate bubbles by tech subsector (Industry) and create industry-level animated bubble chart
# Load subsector mapping and map tickers to Industry
subsector = pd.read_csv('Tech Subsectors - Sheet1.csv')
subsector_map = dict(zip(subsector['Ticker'].str.strip(), subsector['Industry'].str.strip()))

viz_data['Industry'] = viz_data['Company'].map(subsector_map).fillna('Other')

# Aggregate by Date and Industry using market-cap weighted average price
def agg_industry(group):
    total_mc = group['Market_Cap'].sum()
    total_vol = group['Volume'].sum()
    if total_mc > 0:
        avg_price_wgt = np.average(group['Mid_Price'], weights=group['Market_Cap'])
    else:
        avg_price_wgt = group['Mid_Price'].mean()
    return pd.Series({
        'Market_Cap': total_mc,
        'Total_Volume': total_vol,
        'Avg_Price_wgt': avg_price_wgt,
        'Members': group['Company'].nunique()
    })

industry_agg = viz_data.groupby(['Date', 'Industry']).apply(agg_industry).reset_index()
industry_agg['Year_Month'] = industry_agg['Date'].dt.strftime('%Y-%m')

# Create industry-level animated bubble chart
fig = px.scatter(
    industry_agg,
    x='Market_Cap',
    y='Avg_Price_wgt',
    size='Total_Volume',
    color='Industry',
    hover_name='Industry',
    hover_data={
        'Market_Cap': ':,.0f',
        'Avg_Price_wgt': ':.2f',
        'Total_Volume': ':,.0f',
        'Members': True,
        'Year_Month': True
    },
    animation_frame='Year_Month',
    animation_group='Industry',
    size_max=80,
    title='Market Cap vs Avg Price Over Time By Tech Subsector<br><sub>Bubble Size Represents Total Trading Volume</sub>',
    labels={
        'Market_Cap': 'Total Market Capitalization ($, log scale)',
        'Avg_Price_wgt': 'Market-cap-weighted Avg Price ($)',
        'Total_Volume': 'Total Trading Volume',
        'Industry': 'Tech Subsector'
    },
    width=1100,
    height=650
)

# Update layout for better appearance
fig.update_layout(
    title_x=0.5,
    title_font_size=18,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14,
    legend_title_font_size=12,
    showlegend=True,
    legend=dict(
        orientation='v',
        yanchor='top',
        y=1,
        xanchor='left',
        x=1.02
    ),
    xaxis=dict(type='log', gridcolor='lightgray'),
    yaxis=dict(gridcolor='lightgray'),
    plot_bgcolor='white',
    margin=dict(l=80, r=80, t=100, b=80)  # Add padding around the plot
)

# Calculate x-axis range with padding to prevent bubbles from being cut off
min_mc = industry_agg['Market_Cap'].min()
max_mc = industry_agg['Market_Cap'].max()
# For log scale, add padding by multiplying/dividing by a factor (~20% on each side)
x_min = min_mc / 1.5  # Extend left
x_max = max_mc * 1.5  # Extend right

# Calculate y-axis range with padding (linear scale, so add percentage)
min_price = industry_agg['Avg_Price_wgt'].min()
max_price = industry_agg['Avg_Price_wgt'].max()
price_range = max_price - min_price
# Add 10% padding on each side
y_min = min_price - (price_range * 0.1)
y_max = max_price + (price_range * 0.1)

# Update x-axis with expanded range and subtitle
fig.update_xaxes(
    range=[np.log10(x_min), np.log10(x_max)]  # Log scale range in log10 space
)

# Update y-axis with expanded range
fig.update_yaxes(
    range=[y_min, y_max]  # Linear scale range with padding
)

# Tweak animation speed if present
if fig.layout.updatemenus:
    try:
        fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 300
        fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 150
    except Exception:
        pass

fig.update_traces(marker=dict(line=dict(width=1, color='white'), opacity=0.85))

fig.show()


---

## Visualization 2: Tech Sector Performance vs Macroeconomic Indicators

In [17]:
# Prepare macro data
macro['date'] = pd.to_datetime(macro['date'])

# Merge viz_data with macro economic data
viz_with_macro = viz_data.merge(macro, left_on='Date', right_on='date', how='inner')

# Calculate average tech sector performance per month
sector_performance = viz_with_macro.groupby('Date').agg({
    'Mid_Price': 'mean',
    'Real GDP': 'first',
    'Inflation': 'first',
    'Unemployment Rate': 'first',
    'Labor Force Participation': 'first',
    'Nominal GDP': 'first'
}).reset_index()

# Rename for clarity
sector_performance = sector_performance.rename(columns={
    'Mid_Price': 'Tech_Sector_Avg_Price'
})

# Calculate GDP growth rate
sector_performance = sector_performance.sort_values('Date')
sector_performance['GDP_Growth_Rate'] = sector_performance['Real GDP'].pct_change(periods=12) * 100

# Calculate normalized tech sector performance (base 100 at first observation)
first_price = sector_performance['Tech_Sector_Avg_Price'].iloc[0]
sector_performance['Tech_Sector_Performance'] = (sector_performance['Tech_Sector_Avg_Price'] / first_price) * 100

In [18]:
# Function to normalize to 0-100 scale
def normalize_0_100(series):
    """Normalize a series to 0-100 scale"""
    min_val = series.min()
    max_val = series.max()
    if max_val - min_val == 0:
        return series * 0 + 50  # Return 50 if no variation
    return ((series - min_val) / (max_val - min_val)) * 100

# Create normalized dataframe
sector_norm = sector_performance[['Date']].copy()

# Normalize tech sector performance (already base 100, but rescale to 0-100)
sector_norm['Tech_Sector_Performance'] = normalize_0_100(sector_performance['Tech_Sector_Performance'])

# Normalize each economic indicator
indicators = [
    ('Real GDP', 'Real_GDP'),
    ('GDP_Growth_Rate', 'GDP_Growth'),
    ('Inflation', 'Inflation'),
    ('Unemployment Rate', 'Unemployment'),
    ('Labor Force Participation', 'Labor_Force_Participation'),
    ('Nominal GDP', 'Nominal_GDP')
]

for orig_col, new_col in indicators:
    if orig_col in sector_performance.columns:
        sector_norm[new_col] = normalize_0_100(sector_performance[orig_col])

# Remove rows with NaN values
sector_norm = sector_norm.dropna()

In [23]:
# Define indicators to compare (excluding tech performance itself)
indicators_to_plot = [
    ('Real_GDP', 'Real GDP', 'Economic Output'),
    ('GDP_Growth', 'GDP Growth Rate', 'Economic Growth'),
    ('Inflation', 'Core Inflation', 'Price Pressure'),
    ('Unemployment', 'Unemployment Rate', 'Labor Market'),
    ('Labor_Force_Participation', 'Labor Force Participation', 'Labor Market'),
    ('Nominal_GDP', 'Nominal GDP', 'Economic Output')
]

# Filter to only available indicators
available_indicators = [(col, name, cat) for col, name, cat in indicators_to_plot if col in sector_norm.columns]
n_indicators = len(available_indicators)

# Create subplots (3 rows, 2 columns)
n_cols = 3
n_rows = (n_indicators + 1) // 2

fig = make_subplots(
    rows=n_rows,
    cols=n_cols,
    subplot_titles=[name for _, name, _ in available_indicators],
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# Add traces for each indicator
for idx, (col, name, category) in enumerate(available_indicators):
    row = (idx // n_cols) + 1
    col_pos = (idx % n_cols) + 1
    
    # Add tech sector performance line
    fig.add_trace(
        go.Scatter(
            x=sector_norm['Date'],
            y=sector_norm['Tech_Sector_Performance'],
            name='Tech Sector',
            line=dict(color='#d73027', width=2),
            showlegend=(idx == 0)  # Only show legend once
        ),
        row=row,
        col=col_pos
    )
    
    # Add economic indicator line
    fig.add_trace(
        go.Scatter(
            x=sector_norm['Date'],
            y=sector_norm[col],
            name=name,
            line=dict(color='#313695', width=2, dash='dash'),
            showlegend=(idx == 0)  # Only show legend once
        ),
        row=row,
        col=col_pos
    )

# Update layout
fig.update_layout(
    height=400 * n_rows,
    width=1600,
    title_text='Tech Sector Performance vs Macroeconomic Indicators<br><sub>All variables normalized to 0-100 scale for comparison</sub>',
    showlegend=True,
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    ),
    plot_bgcolor='white',
    paper_bgcolor='white'
)

# Update axes
fig.update_xaxes(title_text='Date', gridcolor='lightgray')
fig.update_yaxes(title_text='Normalized Value (0-100)', gridcolor='lightgray', range=[0, 100])

fig.show()

---
## Visualization 3: Event Study Analysis - Economic Crises Impact on Tech Stocks

In [20]:
# Event Study Analysis: Preprocessing for D3 Visualization
# Define economic crisis events
events = {
    'dotcom_bubble': {
        'name': 'Dot-Com Bubble',
        'start_date': '2000-03-01',  
        'end_date': '2002-10-01',
        'window_days': 180  
    },
    'great_recession': {
        'name': 'Great Recession',
        'start_date': '2007-12-01',  
        'end_date': '2009-06-01',
        'window_days': 180
    },
    'covid19': {
        'name': 'COVID-19 Pandemic',
        'start_date': '2020-03-01', 
        'end_date': '2022-12-01',
        'window_days': 180
    }
}

# Calculate returns for all companies
returns_data = viz_data.copy()
returns_data = returns_data.sort_values(['Company', 'Date'])

# Calculate returns - fill NaN with 0 (first row has no previous value, so 0% return)
returns_data['Return'] = returns_data.groupby('Company')['Mid_Price'].pct_change() * 100
returns_data['Return'] = returns_data['Return'].fillna(0)

# Calculate cumulative returns
returns_data['Cumulative_Return'] = returns_data.groupby('Company')['Return'].cumsum()

# Rank companies by average market cap (using latest available data)
latest_rankings = returns_data.groupby('Company')['Market_Cap'].last().sort_values(ascending=False)
company_ranks = {company: rank for rank, company in enumerate(latest_rankings.index, 1)}

# Assign rank groups
def get_rank_group(rank):
    if rank <= 10:
        return 'Top 1-10'
    elif rank <= 20:
        return '11-20'
    elif rank <= 30:
        return '21-30'
    elif rank <= 40:
        return '31-40'
    elif rank <= 50:
        return '41-50'
    else:
        return '51+'

returns_data['Rank'] = returns_data['Company'].map(company_ranks)
returns_data['Rank_Group'] = returns_data['Rank'].apply(get_rank_group)

# Process each event
event_study_data = []

for event_key, event_info in events.items():
    event_start = pd.to_datetime(event_info['start_date'])
    window = event_info['window_days']
    
    # Define window: window_days before to window_days after
    window_start = event_start - pd.Timedelta(days=window)
    window_end = event_start + pd.Timedelta(days=window)
    
    # Filter data for this window
    event_data = returns_data[
        (returns_data['Date'] >= window_start) & 
        (returns_data['Date'] <= window_end)
    ].copy()
    
    if len(event_data) == 0:
        continue
    
    # Calculate days relative to event (0 = event start date)
    event_data['Days_From_Event'] = (event_data['Date'] - event_start).dt.days
    
    # For each company, reset cumulative return to 0 at event start (Day 0)
    # This makes the event study more meaningful - shows performance relative to event start
    for company in event_data['Company'].unique():
        company_mask = event_data['Company'] == company
        company_data = event_data[company_mask].sort_values('Days_From_Event')
        
        # Find the cumulative return at Day 0 (or closest to it)
        day_0_data = company_data[company_data['Days_From_Event'] == 0]
        if len(day_0_data) > 0:
            baseline_return = day_0_data['Cumulative_Return'].iloc[0]
            if not pd.isna(baseline_return):
                # Reset cumulative returns relative to Day 0
                event_data.loc[company_mask, 'Cumulative_Return'] = (
                    event_data.loc[company_mask, 'Cumulative_Return'] - baseline_return
                )
        else:
            # If no Day 0 data, use the first available day as baseline
            if len(company_data) > 0:
                first_return = company_data['Cumulative_Return'].iloc[0]
                if not pd.isna(first_return):
                    event_data.loc[company_mask, 'Cumulative_Return'] = (
                        event_data.loc[company_mask, 'Cumulative_Return'] - first_return
                    )
    
    # For each rank group, calculate average cumulative returns
    for rank_group in ['Top 1-10', '11-20', '21-30', '31-40', '41-50', '51+']:
        group_data = event_data[event_data['Rank_Group'] == rank_group].copy()
        
        if len(group_data) == 0:
            continue
        
        # Remove any NaN values before aggregating
        group_data = group_data.dropna(subset=['Cumulative_Return', 'Days_From_Event'])
        
        if len(group_data) == 0:
            continue
        
        # Aggregate by days from event
        daily_avg = group_data.groupby('Days_From_Event').agg({
            'Cumulative_Return': 'mean',
            'Return': 'mean'
        }).reset_index()
        
        # Ensure no NaN values in aggregated data
        daily_avg = daily_avg.dropna()
        
        # Add metadata
        daily_avg['Event'] = event_key
        daily_avg['Event_Name'] = event_info['name']
        daily_avg['Rank_Group'] = rank_group
        
        event_study_data.append(daily_avg)

# Combine all event data
event_study_df = pd.concat(event_study_data, ignore_index=True)

# Export to JSON for D3
import json


# Global color palette for all visualizations
CUSTOM_COLORS = [
    '#d73027',  # red
    '#f46d43',  # coral
    '#fdae61',  # orange-tan
    '#74add1',  # muted blue
    '#4575b4',  # medium blue
    '#313695'   # deep navy
]

# Prepare data structure for D3
# Use the continuous scale palette from the notebook
d3_data = {
    'events': [],
    'colorPalette': ['#d73027', '#f46d43', '#fdae61', '#74add1', '#4575b4', '#313695']
}

for event_key, event_info in events.items():
    event_data = event_study_df[event_study_df['Event'] == event_key]
    
    if len(event_data) == 0:
        continue
    
    event_obj = {
        'id': event_key,
        'name': event_info['name'],
        'startDate': event_info['start_date'],
        'endDate': event_info['end_date'],
        'series': []
    }
    
    # Group by rank group
    for rank_group in ['Top 1-10', '11-20', '21-30', '31-40', '41-50', '51+']:
        rank_data = event_data[event_data['Rank_Group'] == rank_group].sort_values('Days_From_Event')
        
        # Remove any NaN values before creating series
        rank_data = rank_data.dropna(subset=['Days_From_Event', 'Cumulative_Return'])
        
        if len(rank_data) > 0:
            # Convert to list, ensuring no NaN values
            data_points = rank_data[['Days_From_Event', 'Cumulative_Return']].values.tolist()
            # Filter out any points with NaN
            data_points = [[float(d[0]), float(d[1])] for d in data_points 
                          if not (pd.isna(d[0]) or pd.isna(d[1]))]
            
            if len(data_points) > 0:
                series = {
                    'name': rank_group,
                    'data': data_points
                }
                event_obj['series'].append(series)
    
    d3_data['events'].append(event_obj)

# Save to JSON file
with open('viz_5/event_study_data.json', 'w') as f:
    json.dump(d3_data, f, indent=2)